# 4-6 랜덤 포레스트

### 랜덤 포레스트란?

데이터를 기반으로 다수의 의사결정 트리를 만들고 각 트리를 기반으로 다수결로 결과를 정하여 예측, 학습 데이터를 무작위로 샘플링해서 만들어진 다수의 의사결정 트리를 사용하기 때문에 랜덤 포레스트라고 명명.

주요 Point

- 샘플링 할 때 sub-sample 사이즈는 항상 input sample 사이즈와 같음. (bootstrap sample이라고 하며 [1,2,3,4,5] -> [2,2,4,3,1]와 같이 샘플링 합니다)
- 각 특징(예를들어 버섯의 색깔, 형태. 냄새 등..)들도 sampling 하여 (보통 sqrt(N)개 한다고 합니다.) 여러개의 트리를 생성
- 각 트리별 예측을 보고 다수결에 따라 예측값 결정

![](.\1.png)

참고자료
 - scikit-learn : Randomforestclassifier http://scikit-learn.org/stable/modules/ensemble.html

 - 네이버 블로그 : http://sams.epaiai.com/220979751089

### 랜덤 포레스트 사용하기 : 버섯 분류(독버섯과 식용버섯)


- 독버섯 데이터셋 저장 : url로 부터 독버섯 데이터를 불러와 mushroom.csv로 저장합니다.

In [91]:
import urllib.request as req
local= "mushroom.csv"
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data"
req.urlretrieve(url, local)
print("ok")

ok


- 랜덤 포레스트를 사용한 독버섯 분류 예제 1

In [47]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [48]:
# 데이터 읽어 들이기--- (※1)
mr = pd.read_csv("mushroom.csv", header=None)

In [80]:
print('mr = ', mr)

In [49]:
# 데이터 내부의 기호를 숫자로 변환하기--- (※2)
label = []
data = []
attr_list = []
for row_index, row in mr.iterrows():
    label.append(row.loc[0])
    row_data = []
    for v in row.iloc[1:]:
        row_data.append(ord(v))  # 참고 : ord() 는 문자의 아스키 값을 반환하여 줍니다. a ~ z -> 97 ~ 122, ? -> 63
    data.append(row_data)

In [65]:
print('label = ', label)

In [66]:
print('data = ',data)

In [18]:
# 학습 전용과 테스트 전용 데이터로 나누기 --- (※3)
data_train, data_test, label_train, label_test = \
    train_test_split(data, label) # train , test 비율을 3:1 비율로 나누어 줍니다.

In [84]:
print('data_train의 수 : ', len(data_train))
print('data_test의 수 : ', len(data_test))

In [19]:
# 데이터 학습시키기 --- (※4)
clf = RandomForestClassifier()
clf.fit(data_train, label_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

### RandomForestClassifier class
#### class sklearn.ensemble.RandomForestClassifier(n_estimators=10, criterion=’gini’, max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=’auto’, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=1, random_state=None, verbose=0, warm_start=False, class_weight=None)


- max_features : int, float, string or None, optional (default=”auto”) , If “auto”, then max_features=sqrt(n_features).
- bootstrap : boolean, optional (default=True), Whether bootstrap samples are used when building trees.

In [22]:
# 데이터 예측하기 --- (※5)
predict = clf.predict(data_test)
# 결과 테스트하기 --- (※6)
ac_score = metrics.accuracy_score(label_test, predict)
cl_report = metrics.classification_report(label_test, predict)
print("정답률 =", ac_score)
print("리포트 =\n", cl_report)

정답률 = 1.0
리포트 =
              precision    recall  f1-score   support

          e       1.00      1.00      1.00      1077
          p       1.00      1.00      1.00       954

avg / total       1.00      1.00      1.00      2031



## 데이터를 숫자로 변경할 때 주의할 사항

버섯의 색을
빨강 = 1, 파랑 = 2, 초록 = 3, 흰색 =4 와 같이 숫자를 할당하였을 때, 각각의 데이터가 상관이 없다는것을 나타내기 위해서는

트레이닝(FIT)전에 아래와 같이 데이터를 변경해주어야 합니다.

빨강 1 0 0 0
파랑 0 1 0 0
초록 0 0 1 0
흰색 0 0 0 1

이것을 "one hot" encoding 이라고 합니다. [변경된 표현방식을 보면 하나의 요소만 1로 표현('hot')되어있음]
대부분의 머신러닝에서 흔히 쓰이며 독버섯 분류 예제 모델 1에서 직접 각 특징들을 one hot encoding 해주지 않았지만 fit 이라는 내장함수 안에 one hot encoding이 포함되어 있습니다.

- 랜덤 포레스트를 사용한 독버섯 분류 예제 2 : one hot encoding 직접 해보기

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [ ]:
# 데이터 읽어 들이기
mr = pd.read_csv("mushroom.csv", header=None)

In [76]:
# 데이터 내부의 분류 변수 전개하기
label = []
data = []
attr_list = []
for row_index, row in mr.iterrows():
    label.append(row.loc[0])
    exdata = []
    for col, v in enumerate(row.iloc[1:]):   # enumerate : 값과 index를 반환할 때 사용
        if row_index == 0:
            attr = {"dic": {}, "cnt":0}
            attr_list.append(attr)
        else:
            attr = attr_list[col]
        # 버섯의 특징 기호를 배열로 나타내기
        d = [0,0,0,0,0,0,0,0,0,0,0,0]    # 하나의 특징을 총 12개의 요소를 가지는 배열로 one hot encoding 합니다. (22 * 12)
        if v in attr["dic"]:
            idx = attr["dic"][v]
        else:
            idx = attr["cnt"]
            attr["dic"][v] = idx
            attr["cnt"] += 1
        d[idx] = 1
        exdata += d
    data.append(exdata)

In [89]:
print('labe = ', label)
print('data 수 = ', len(data))
print('data 중 첫번째 특징 = ', len(data[0]))
print(data[0])

In [33]:
# 학습 전용 데이터와 테스트 전용 데이터로 나누기
data_train, data_test, label_train, label_test = \
train_test_split(data, label) #3:1 로 나누어줍니다.

In [34]:
# 데이터 학습시키기
clf = RandomForestClassifier()
clf.fit(data_train, label_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [35]:
# 데이터 예측하기
predict = clf.predict(data_test)
# 결과 테스트하기
ac_score = metrics.accuracy_score(label_test, predict)
print("정답률 =", ac_score)

정답률 = 1.0
